# Title

Short description of what this notebook is about

## Preliminaries

In [1]:
# To auto-reload modules in jupyter notebook (so that changes in files *.py doesn't require manual reloading):
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
%load_ext autoreload
%autoreload 2

Import commonly used libraries and magic command for inline plotting

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
%matplotlib inline

## Work

This is where work gets done.

Here's a high-level outline of the notebook:

Introduction and Setup

    Introduction to the project.
    Import necessary libraries (SimPy, pandas, matplotlib, etc.).

Define the Simulation Environment

    Create the environment.
    Define resources such as staff and treatment rooms.


Define Processes

    Patient arrival.
    Registration process.
    Triage process.
    Treatment process.
    Discharge process.

Run the Simulation

    Generate patient arrivals.
    Collect data on patient wait times and system throughput.

Analyze the Results

    Calculate key metrics (average wait time, time in system, etc.).
    Plot histograms and summary statistics.

In [ ]:
# Import necessary libraries
import simpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define the simulation environment and resources
class EmergencyDepartment:
    def __init__(self, env, num_doctors, num_nurses, num_beds):
        self.env = env
        self.doctor = simpy.Resource(env, num_doctors)
        self.nurse = simpy.Resource(env, num_nurses)
        self.bed = simpy.Resource(env, num_beds)

# Define patient processes
def patient(env, name, ed):
    arrival_time = env.now

    # Registration
    with ed.nurse.request() as request:
        yield request
        yield env.timeout(np.random.uniform(5, 10))  # Registration takes 5-10 minutes

    # Triage
    with ed.nurse.request() as request:
        yield request
        yield env.timeout(np.random.uniform(5, 15))  # Triage takes 5-15 minutes

    # Treatment
    with ed.doctor.request() as request:
        yield request
        with ed.bed.request() as request:
            yield request
            yield env.timeout(np.random.uniform(30, 120))  # Treatment takes 30-120 minutes

    # Discharge
    discharge_time = env.now
    patient_log.append([name, arrival_time, discharge_time])

# Setup and run the simulation
def run_simulation(num_patients, num_doctors, num_nurses, num_beds, simulation_time):
    global patient_log
    patient_log = []

    env = simpy.Environment()
    ed = EmergencyDepartment(env, num_doctors, num_nurses, num_beds)

    for i in range(num_patients):
        env.process(patient(env, f'Patient {i+1}', ed))
        yield env.timeout(np.random.exponential(10))  # New patient arrives every 10 minutes on average

    env.run(until=simulation_time)

# Parameters
num_patients = 100
num_doctors = 3
num_nurses = 5
num_beds = 10
simulation_time = 480  # 8 hours

# Run the simulation
run_simulation(num_patients, num_doctors, num_nurses, num_beds, simulation_time)

# Analyze the results
patient_log_df = pd.DataFrame(patient_log, columns=['Patient', 'Arrival', 'Discharge'])
patient_log_df['Time in System'] = patient_log_df['Discharge'] - patient_log_df['Arrival']

# Summary statistics
print(patient_log_df.describe())

# Plot histograms
plt.hist(patient_log_df['Time in System'], bins=20)
plt.xlabel('Time in System (minutes)')
plt.ylabel('Number of Patients')
plt.title('Distribution of Time in System')
plt.show()


I'll create a similar project but instead of a vaccine clinic, I'll model a hospital emergency department (ED). The ED will include processes such as patient triage, consultation with doctors, and treatment. The resources will include triage nurses, doctors, and treatment rooms.

Hospital Emergency Department Simulation
Here's an outline of the new project:

Define Resources:

Triage nurses
Doctors
Treatment rooms
Processes:

Patient arrival
Triage
Consultation
Treatment
Parameters:

Arrival rate of patients
Service times for each process
Number of resources

In [ ]:
import simpy
import random
import pandas as pd
from numpy.random import default_rng

class EmergencyDepartment:
    def __init__(self, env, num_triage_nurses, num_doctors, num_treatment_rooms, rng):
        self.env = env
        self.triage_nurses = simpy.Resource(env, num_triage_nurses)
        self.doctors = simpy.Resource(env, num_doctors)
        self.treatment_rooms = simpy.Resource(env, num_treatment_rooms)
        self.rng = rng
        self.waiting_times = []
    
    def triage(self, patient):
        triage_time = self.rng.uniform(5, 15)
        yield self.env.timeout(triage_time)
    
    def consult(self, patient):
        consult_time = self.rng.uniform(10, 30)
        yield self.env.timeout(consult_time)
    
    def treat(self, patient):
        treatment_time = self.rng.uniform(20, 60)
        yield self.env.timeout(treatment_time)

def patient_process(env, ed, patient_id, rng):
    arrival_time = env.now
    
    with ed.triage_nurses.request() as request:
        yield request
        yield env.process(ed.triage(patient_id))
    
    with ed.doctors.request() as request:
        yield request
        yield env.process(ed.consult(patient_id))
    
    with ed.treatment_rooms.request() as request:
        yield request
        yield env.process(ed.treat(patient_id))
    
    waiting_time = env.now - arrival_time
    ed.waiting_times.append(waiting_time)

def run_ed(env, ed, mean_interarrival_time, rng, stoptime):
    patient_id = 0
    while env.now < stoptime:
        interarrival_time = rng.exponential(mean_interarrival_time)
        yield env.timeout(interarrival_time)
        patient_id += 1
        env.process(patient_process(env, ed, patient_id, rng))

def main():
    # Parameters
    patients_per_hour = 10
    mean_interarrival_time = 60.0 / patients_per_hour
    num_triage_nurses = 3
    num_doctors = 2
    num_treatment_rooms = 4
    stoptime = 480  # 8 hours
    
    # Create a random number generator
    rng = default_rng(seed=12345)
    
    # Create a simulation environment
    env = simpy.Environment()
    
    # Create an emergency department
    ed = EmergencyDepartment(env, num_triage_nurses, num_doctors, num_treatment_rooms, rng)
    
    # Register the run_ed (generator) function
    env.process(run_ed(env, ed, mean_interarrival_time, rng, stoptime=stoptime))
    
    # Run the simulation
    env.run()
    
    # Output results
    waiting_times_df = pd.DataFrame(ed.waiting_times, columns=["Waiting Time"])
    print(waiting_times_df.describe())
    waiting_times_df.to_csv("ed_waiting_times.csv", index=False)

if __name__ == "__main__":
    main()
